In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

!pip install wandb --quiet
!pip install python-Levenshtein -q

!pip install torchsummaryX -q
!pip install mne -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 726.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 40.6 MB/s eta 0:00:00


In [2]:
import torch
import torchaudio
from torch import nn, Tensor
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# import torchsummary

import numpy as np
import os

import gc
import time

import pandas as pd
from tqdm.notebook import tqdm as blue_tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import json

import math
from typing import Optional, List


#imports for decoding and distance calculation
try:
    import wandb
    import torchsummaryX
    import Levenshtein
except:
    print("Didnt install some/all imports")

import sklearn
import mne

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("Device: ", device)

Device:  cuda


In [3]:
config = dict (
    batch_size          = 32,
    epochs              = 100,
    learning_rate       = 1e-3, # per write up
    weight_decay        = 0,
)

# Suggested transforms per the write up
transforms = torch.nn.Sequential(
    torchaudio.transforms.TimeMasking(time_mask_param=225),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=6)
)

In [4]:
VOCAB = [
    '<pad>', '<sos>', '<eos>',
    'A',   'B',    'C',    'D',
    'E',   'F',    'G',    'H',
    'I',   'J',    'K',    'L',
    'M',   'N',    'O',    'P',
    'Q',   'R',    'S',    'T',
    'U',   'V',    'W',    'X',
    'Y',   'Z',    "'",    ' ',
]

VOCAB_MAP = {VOCAB[i]:i for i in range(0, len(VOCAB))}

PAD_TOKEN = VOCAB_MAP["<pad>"]
SOS_TOKEN = VOCAB_MAP["<sos>"]
EOS_TOKEN = VOCAB_MAP["<eos>"]

print(f"Length of vocab : {len(VOCAB)}")
print(f"Vocab           : {VOCAB}")
print(f"PAD_TOKEN       : {PAD_TOKEN}")
print(f"SOS_TOKEN       : {SOS_TOKEN}")
print(f"EOS_TOKEN       : {EOS_TOKEN}")

Length of vocab : 31
Vocab           : ['<pad>', '<sos>', '<eos>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', "'", ' ']
PAD_TOKEN       : 0
SOS_TOKEN       : 1
EOS_TOKEN       : 2


In [5]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


In [6]:
# Load in the new and glorious Phoneme-added dataframe
import pandas as pd
data_csv =pd.read_csv("drive/MyDrive/brennan data/AliceChapterOne-EEG-Phonemes.csv")
data_csv.head()

,Unnamed: 0,Word,Segment,onset,offset,Order,LogFreq,LogFreq_Prev,LogFreq_Next,SndPower,Length,Position,Sentence,IsLexical,NGRAM,RNN,CFG,phonemes
0,0,Alice,1,0.046000,0.608721,1,8.65,0.00,14.56,3.620000e-07,0.562721,1,1,1.0,3.226499,3.126175,2.312348,"['AE1', 'L', 'IH0', 'S']"
1,1,was,1,0.562721,0.830543,2,14.56,8.65,10.69,3.840000e-09,0.267822,2,1,0.0,0.905229,1.691128,1.357460,"['W', 'AH0', 'Z']"
2,2,beginning,1,0.784543,1.302929,3,10.69,14.56,16.35,3.690000e-09,0.518386,3,1,1.0,4.446766,4.100771,5.626722,"['B', 'IH0', 'G', 'IH1', 'N', 'IH0', 'NG']"
3,3,to,1,1.256929,1.398925,4,16.35,10.69,13.79,3.970000e-09,0.141996,4,1,0.0,2.537495,3.833313,5.939201,"['T', 'AH0']"
4,4,get,1,1.352925,1.662327,5,13.79,16.35,13.28,3.770000e-09,0.309402,5,1,0.0,1.023137,1.013076,2.697304,"['G', 'IH1', 'T']"


In [7]:
TRANSCRIPTS = []

for i in range(len(np.unique(data_csv.Segment))):
  words = " ".join(data_csv["Word"].loc[data_csv.Segment == i+1].values.tolist())
  words = "".join([i.upper() for i in words])
  TRANSCRIPTS.append(words)

print(TRANSCRIPTS)

['ALICE WAS BEGINNING TO GET VERY TIRED OF SITTING BY HER SISTER ON THE BANK AND OF HAVING NOTHING TO DO ONCE OR TWICE SHE PEEPED INTO THE BOOK HER SISTER WAS READING BUT IT HAD NO PICTURES OR CONVERSATIONS IN IT AND WHAT THE USE OF A BOOK THOUGHT ALICE WITHOUT PICTURES OR CONVERSATION SO SHE WAS CONSIDERING IN HER OWN MIND AS WELL AS SHE COULD FOR THE HOT DAY MADE HER FEEL VERY SLEEPY AND STUPID WHETHER THE PLEASURE OF MAKING A DAISY CHAIN WOULD BE WORTH THE TROUBLE OF GETTING UP AND PICKING THE DAISIES WHEN SUDDENLY A WHITE RABBIT WITH PINK EYES RAN CLOSE BY HER THERE WAS NOTHING SO VERY REMARKABLE IN THAT NOR DID ALICE THINK IT SO VERY MUCH OUT OF THE WAY TO HEAR THE RABBIT SAY TO ITSELF OH DEAR OH DEAR I SHALL BE LATE WHEN SHE THOUGHT IT OVER AFTERWARDS IT OCCURRED TO HER THAT SHE OUGHT TO HAVE WONDERED AT THIS BUT AT THE TIME IT ALL SEEMED QUITE NATURAL', 'BUT WHEN THE RABBIT ACTUALLY TOOK A WATCH OUT OF ITS POCKET AND LOOKED AT IT AND THEN HURRIED ON ALICE STARTED TO HER FEET FOR

In [8]:
class EEGSpeechDatasetME(torch.utils.data.Dataset): # Memory efficient
    # Loades the data in get item to save RAM

    def __init__(self, EEG_data, transcripts = TRANSCRIPTS):

        self.VOCAB      = VOCAB

        self.transcripts_files = TRANSCRIPTS

        assert len(EEG_data) == len(transcripts)

        length = len(mfcc_files) # TODO

        self.EEG_files          =  EEG_data
        self.length             = len(self.EEG_files)

    def __len__(self):
        # TODO
        return self.length

    def __getitem__(self, ind):

        # Load the mfcc and transcripts from the mfcc and transcript paths created earlier
        EEG        = self.EEG_files[ind]  # TODO

        transcript  = self.transcript_files[ind] # TODO
        transcript_mapped   = [SOS_TOKEN] + [VOCAB_MAP[i] for i in transcript] + [EOS_TOKEN]# TODO
        transcript_mapped =  np.array(transcript_mapped)

        return torch.FloatTensor(EEG), torch.LongTensor(transcript_mapped)


    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # TODO
            batch_x.append(x)
            batch_y.append(y)
            lengths_x.append(len(x))
            lengths_y.append(len(y))

        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        batch_x_pad = pad_sequence(batch_x, batch_first=True, padding_value=PAD_TOKEN)# TODO
        batch_y_pad = pad_sequence(batch_y, batch_first=True, padding_value=PAD_TOKEN)# TODO

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

In [9]:
import glob
from collections import Counter

vhdr_files = glob.glob("drive/MyDrive/brennan data/*/*.vhdr") # only use five? smaller dataset, more neurons??

In [10]:
  eeg_data_list = []
  mne.set_log_level('WARNING')

  for num, vhdr_file in enumerate(vhdr_files):
    print(f"Loading {num + 1} of {len(vhdr_files)}")

    # Step 1: Load the file
    mne_raw = mne.io.read_raw_brainvision(vhdr_file, misc=['ECG', 'EMG', 'FootPad'],preload=True)

    # A couple of the subjects have a different number of channels.
    # As someone who does EEG data research, I get it.
    # As someone doing this class project, I hate them.
    # Let's just ignore those people with different cap sizes to simplify things
    if mne_raw.get_data().shape[0] == 62:

      # Step 2: Preprocess the filter
      # Note: This is very minimal, but we might not want to do too much more

      # Bandpass filter 0.5 - 40 Hz
      mne_raw = mne_raw.filter(0.5, 40);

      # Common Average Reference
      mne_raw.set_eeg_reference('average', projection=False);

      # Resampling to 100 Hz is done AFTER segmentation into words to not lose
      # time resolution when cropping (see: .resample(100) below)

      # Step 3: Segment into the 12 time-stamped segments
      # For some reason, some of the dataset first stimulus is called "stimulus 1"
      # and other times is new segment -.- . Also, calling events from annotations
      # can return the list in unsorted order, which will completely mess up the
      # whole labels to eeg alignment we have going on.
      # So,
      # mne_events, ev_id = mne.events_from_annotations(
      #           raw=mne_raw, event_id={"Stimulus/{:d}".format(i): i for i in range(0,13)})

      mne_events, ev_id = mne.events_from_annotations(raw=mne_raw, event_id=Counter(mne_raw.annotations.description))
      # because strings are weird, this actually returns the events out of order.
      # so, let's sort them according to their timestamp. Easy peasy!
      mne_events = sorted(mne_events, key = lambda x: x[0])


      if len(mne_events) > 12:
        # Only keep last 12 if there was an extra starting stimulus,
        # Otherwise, consider the starting stimulus as start of sequence
        mne_events = mne_events[-12:]

      # Crop each segment and save it seperately
      segments = [mne_raw.copy().crop(mne_events[i][0] * 1/mne_raw.info['sfreq'], mne_events[i+1][0] * 1/mne_raw.info['sfreq']).resample(100) for i in range(len(mne_events) - 1)]
      segments.append(mne_raw.copy().crop(mne_events[-1][0] * 1/mne_raw.info['sfreq'], None).resample(100))
      eeg_data_list.extend(segments)
      # print(eeg_data_list)

Loading 1 of 49
Loading 2 of 49
Loading 3 of 49
Loading 4 of 49
Loading 5 of 49
Loading 6 of 49
Loading 7 of 49
Loading 8 of 49
Loading 9 of 49
Loading 10 of 49
Loading 11 of 49
Loading 12 of 49
Loading 13 of 49
Loading 14 of 49
Loading 15 of 49
Loading 16 of 49
Loading 17 of 49
Loading 18 of 49
Loading 19 of 49
Loading 20 of 49
Loading 21 of 49
Loading 22 of 49
Loading 23 of 49
Loading 24 of 49
Loading 25 of 49
Loading 26 of 49
Loading 27 of 49
Loading 28 of 49
Loading 29 of 49
Loading 30 of 49
Loading 31 of 49
Loading 32 of 49
Loading 33 of 49
Loading 34 of 49
Loading 35 of 49
Loading 36 of 49
Loading 37 of 49
Loading 38 of 49
Loading 39 of 49
Loading 40 of 49
Loading 41 of 49
Loading 42 of 49
Loading 43 of 49
Loading 44 of 49
Loading 45 of 49
Loading 46 of 49
Loading 47 of 49
Loading 48 of 49
Loading 49 of 49


In [11]:
# DATA_DIR    = "drive/MyDrive/brennan data/S01" # TODO: Path where you have downloaded the data

# mne_raw = mne.io.read_raw_brainvision(os.path.join(DATA_DIR, "S01.vhdr"),
                                      # misc=['ECG', 'EMG', 'FootPad'],preload=True)

In [12]:
# # You'll notice in the dataframe that they've timestamped each word with regard to
# # the segmented file! That was very nice of them. Let's extract the word onset/offset
# import numpy as np
# segment_onset_offset = []
# for seg_val in np.unique(data_csv["Segment"].values):
#   seg = data_csv.loc[data_csv.Segment == seg_val]
#   onsets = seg["onset"].values
#   offsets = seg["offset"].values
#   # Append to onset/offset times
#   segment_onset_offset.append(list(zip(onsets, offsets)))


In [13]:
# # Now its time to load in the EEG data
# # For this, we have to loop through all the subjects (49) and extract
# # each word based on the segment and timestamp
# import mne
# import glob
# from collections import Counter

# if wannaSpendForeverMakingDataAgain:

#   vhdr_files = glob.glob("drive/MyDrive/brennan data/*/*.vhdr")

#   vhdr_files

#   eeg_data_list = []
#   labels_list = []


#   def data_sorter(data_string):
#       try:
#           return int(data_string.split(r"/")[-1])
#       except:
#           # handle division by zero error
#           # leave empty for now
#           return 0

#   for vhdr_file in vhdr_files[26:]:

#     # Step 1: Load the file
#     mne_raw = mne.io.read_raw_brainvision(vhdr_file, misc=['ECG', 'EMG', 'FootPad'],preload=True)


#     # A couple of the subjects have a different number of channels.
#     # As someone who does EEG data research, I get it.
#     # As someone doing this class project, I hate them.
#     # Let's just ignore those people with different cap sizes to simplify things
#     if mne_raw.get_data().shape[0] == 62:

#       # Step 2: Preprocess the filter
#       # Note: This is very minimal, but we might not want to do too much more

#       # Bandpass filter 0.5 - 40 Hz
#       mne_raw = mne_raw.filter(0.5, 40);

#       # Common Average Reference
#       mne_raw.set_eeg_reference('average', projection=False);

#       # Resampling to 100 Hz is done AFTER segmentation into words to not lose
#       # time resolution when cropping (see: .resample(100) below)

#       # Step 3: Segment into the 12 time-stamped segments
#       # For some reason, some of the dataset first stimulus is called "stimulus 1"
#       # and other times is new segment -.- . Also, calling events from annotations
#       # can return the list in unsorted order, which will completely mess up the
#       # whole labels to eeg alignment we have going on.
#       # So,
#       # mne_events, ev_id = mne.events_from_annotations(
#       #           raw=mne_raw, event_id={"Stimulus/{:d}".format(i): i for i in range(0,13)})

#       mne_events, ev_id = mne.events_from_annotations(raw=mne_raw, event_id=Counter(mne_raw.annotations.description))
#       # because strings are weird, this actually returns the events out of order.
#       # so, let's sort them according to their timestamp. Easy peasy!
#       mne_events = sorted(mne_events, key = lambda x: x[0])

#       if len(mne_events) > 12:
#         # Only keep last 12 if there was an extra starting stimulus,
#         # Otherwise, consider the starting stimulus as start of sequence
#         mne_events = mne_events[-12:]

#       # We're just going to crop at onset time. This is because the eeg timestamps sometimes overlap
#       # into the next segment. *eye roll*
#       segments = [mne_raw.copy().crop(mne_events[i][0] * 1/mne_raw.info['sfreq'], None) for i in range(len(mne_events))]

#       # Step 4: Further break down each segment into the individual words
#       for i in range(len(segments)): # there should be 12extracted_sentences = [sentences[0].copy().crop(timestamps[i], timestamps[i+1]) for i in range(len(timestamps) - 1)]
#         #print("Segment: ", i)
#         extracted_words = [segments[i].copy().crop(segment_onset_offset[i][j][0], segment_onset_offset[i][j][1]).resample(100) for j in range(len(segment_onset_offset[i]))]
#         eeg_data_list.extend(extracted_words)
#         #print(eeg_data_list)


In [14]:

#targets = data_csv['phonemes']
#phoneme_list = phoneme_list * len(vhdr_files) # Repeat the labels for each subject

num_subj = len(eeg_data_list) // 12 # 12 sets per subject
targets = []
[targets.extend(TRANSCRIPTS) for _ in range(num_subj)]
print(len(eeg_data_list), len(targets))


564 564


In [15]:
# split into training, testing, and validation set
import sklearn
from sklearn import model_selection

train_targets, val_targets, train_eeg, val_eeg = sklearn.model_selection.train_test_split(targets, eeg_data_list, test_size=0.20, random_state=42) # 80/20 train-val/test split
# train_targets, val_targets, train_eeg, val_eeg = sklearn.model_selection.train_test_split(train_val_targets, train_val_eeg, test_size=0.20, random_state=42) # 80/20 train / val split

In [16]:
# import torch

# class Dataset(torch.utils.data.Dataset):

#     def __init__(self, input_data, # EEG data
#                  output_data = None, # phoneme data
#                  mode = 'Training',
#                  phonemes = PHONEMES,
#                  min_length = 0,
#                 ): # Feel free to add more arguments

#         """
#         Input:
#         input_data: Preprocessed EEG data of array of shape (N_samples, ), where each sample is shape (n_time, n_channels),
#                     where n_channels is a consistent number across all samples
#         output_data: phoneme data of read transcript shape (N_samples, ), where each sample is shape (n_phonemes)

#         """
#         assert mode in ['Training', 'Testing', 'Validation'], "Error! Unrecognized Mode"

#         if mode != "Testing":
#           data_tuple = [(i, j) for i,j in zip(input_data, output_data) if i.shape[0] > min_length]
#           input_data = [i[0] for i in data_tuple]
#           output_data = [i[1] for i in data_tuple]
#           print("Input data len: ", len(input_data))
#           assert len(input_data) == len(output_data)

#         else:
#           input_data = [i for i in input_data if i.shape[0] > min_length]


#         self.EEG = input_data
#         self.length = len(self.EEG)
#         self.mode = mode

#         if mode != 'Testing':
#             self.transcripts = []
#             for transcript in output_data: #transcript is one word
#                 # print(transcript)
#                 transcript_list = []
#                 for phoneme in transcript:
#                   phoneme = ''.join([i for i in phoneme if not i.isdigit()])
#                   transcript_list.append(phonemes.index(phoneme))

#                 self.transcripts.append(transcript_list)
#         #print(self.transcripts)

#     def __len__(self):
#         return self.length

#     def __getitem__(self, ind):

#         frames = self.EEG[int(ind)]
#         frames = torch.FloatTensor(frames) # Convert to tensors

#         if self.mode in ["Training", "Validation"]:
#             #labels = torch.tensor(self.transcripts, dtype=torch.long)
#             labels    = torch.tensor(self.transcripts[ind], dtype=torch.long)
#             return frames, labels

#         else:
#             return frames

#     def collate_fn(self, batch):
#         '''
#         TODO:
#         1.  Extract the features and labels from 'batch'
#         2.  We will additionally need to pad both features and labels,
#             look at pytorch's docs for pad_sequence
#         3.  This is a good place to perform transforms, if you so wish.
#             Performing them on batches will speed the process up a bit.
#         4.  Return batch of features, labels, lenghts of features,
#             and lengths of labels.
#         '''
#         # batch of input mfcc coefficients
#         if self.mode in ["Training", "Validation"]:
#             batch_EEG =  [x[0] for x in batch]
#             lengths_EEG = [len(x[0]) for x in batch]# TODO
#             batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

#             #batch_transcript = [x[0][1] for x in batch] # TODO
#             batch_transcript = [x[1] for x in batch]

#             lengths_transcript = [len(x[1]) for x in batch] # TODO
#             #print('lengths_transcript is ', lengths_transcript)
#             #batch_transcript =  torch.tensor(batch_transcript)


#             # Batch x Time x 62 channels
#             batch_transcript_pad = pad_sequence(batch_transcript,
#                                                 batch_first=True,
#                                                 padding_value=0.0) # TODO

#             return batch_EEG_pad, batch_transcript_pad, torch.tensor(lengths_EEG), torch.tensor(lengths_transcript)

#         else:
#             batch_EEG =  [x for x in batch]
#             lengths_EEG = [len(x) for x in batch]# TODO
#             batch_EEG_pad = pad_sequence(batch_EEG, batch_first=True, padding_value=0.0)

#             return batch_EEG_pad, torch.tensor(batch_EEG)

In [17]:
class EEGSpeechDatasetME(torch.utils.data.Dataset): # Memory efficient
    # Loades the data in get item to save RAM

    def __init__(self, EEG_data, transcripts):

        self.VOCAB      = VOCAB
        self.transcript_files = transcripts

        assert len(EEG_data) == len(transcripts)

        self.EEG_files          =  EEG_data
        self.length             =  len(self.EEG_files)

    def __len__(self):
        # TODO
        return self.length

    def __getitem__(self, ind):

        # Load the mfcc and transcripts from the mfcc and transcript paths created earlier
        EEG        = self.EEG_files[ind]  # TODO

        transcript  = self.transcript_files[ind] # TODO
        transcript_mapped   = [SOS_TOKEN] + [VOCAB_MAP[i] for i in transcript] + [EOS_TOKEN] # TODO
        transcript_mapped =  np.array(transcript_mapped)

        return torch.FloatTensor(EEG), torch.LongTensor(transcript_mapped)


    def collate_fn(self,batch):

        batch_x, batch_y, lengths_x, lengths_y = [], [], [], []

        for x, y in batch:
            # Add the mfcc, transcripts and their lengths to the lists created above
            # TODO
            batch_x.append(x)
            batch_y.append(y)
            lengths_x.append(len(x))
            lengths_y.append(len(y))

        # pack the mfccs and transcripts using the pad_sequence function from pytorch
        batch_x_pad = pad_sequence(batch_x, batch_first=True, padding_value=PAD_TOKEN)# TODO
        batch_y_pad = pad_sequence(batch_y, batch_first=True, padding_value=PAD_TOKEN)# TODO

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

In [18]:
# Silence the logs because otherwise its just too much

mne.set_log_level(False)

def bandpass_filter_and_stack(mne_raw, all = True):
  if not all:
    theta = mne_raw.copy().filter(4, 8).get_data().T
    alpha = mne_raw.copy().filter(8, 12).get_data().T
    beta = mne_raw.copy().filter(12, 20).get_data().T
    gamma = mne_raw.copy().filter(20, 40).get_data().T

    stacked = np.stack([theta, alpha, beta, gamma], axis = 1)
    return stacked
  else:
    all = mne_raw.copy().filter(4, 8).get_data().T
    return all



In [19]:
len(train_targets)

451

In [20]:
train_data = EEGSpeechDatasetME(EEG_data = [bandpass_filter_and_stack(i) for i in train_eeg], # .get_data().T for i in range(0, 4)],
                     transcripts = train_targets)

val_data = EEGSpeechDatasetME(EEG_data = [bandpass_filter_and_stack(i) for i in val_eeg], # [extracted_sentences[4].get_data().T],
                     transcripts = val_targets)


# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2, # <--- probably want to increase this :)
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config["batch_size"],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = train_data.collate_fn
)

# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data,
#     num_workers = 2,
#     batch_size  = 2,  # not a lot of samples . . .
#     pin_memory  = True,
#     shuffle     = False,
#     collate_fn  = train_data.collate_fn
# )

In [21]:
# # sanity check
# print("Batch size: ", config["batch_size"])
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))

# for i, data in enumerate(train_loader):
#     x, y, lx, ly = data
#     print(x.shape, y.shape, lx.shape, ly.shape)
#     print(y)
#     break

In [22]:
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

def plot_attention(attention):
    # Function for plotting attention
    # You need to get a diagonal plot
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()


def save_model(model, optimizer, scheduler, tf_scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'tf_scheduler'             : tf_scheduler,
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(best_path, epoch_path, model, mode= 'best', metric= 'valid_acc', optimizer= None, scheduler= None, tf_scheduler= None):


    if mode == 'best':
        checkpoint  = torch.load(best_path)
        print("Loading best checkpoint: ", checkpoint[metric])
    else:
        checkpoint  = torch.load(epoch_path)
        print("Loading epoch checkpoint: ", checkpoint[metric])

    model.load_state_dict(checkpoint['model_state_dict'], strict= False)

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        #optimizer.param_groups[0]['lr'] = 1.5e-3
        optimizer.param_groups[0]['weight_decay'] = 1e-5
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    if tf_scheduler != None:
        tf_scheduler    = checkpoint['tf_scheduler']

    epoch   = checkpoint['epoch']
    metric  = torch.load(best_path)[metric]

    return [model, optimizer, scheduler, tf_scheduler, epoch, metric]

class TimeElapsed():
    def __init__(self):
        self.start  = -1

    def time_elapsed(self):
        if self.start == -1:
            self.start = time.time()
        else:
            end = time.time() - self.start
            hrs, rem    = divmod(end, 3600)
            min, sec    = divmod(rem, 60)
            min         = min + 60*hrs
            print("Time Elapsed: {:0>2}:{:02}".format(int(min),int(sec)))
            self.start  = -1

In [23]:
class pBLSTM(torch.nn.Module):
    """
    The pyramidal bidirectional lstm takes in a sequence of packed x and downsamples it in time by a factor of two
    and upsamples its number of features by two.

    """

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        # Initialize bidirectional LSTM
        self.lstm = torch.nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                                  num_layers = 1, bidirectional = True, batch_first = True)


    def forward(self, x_packed, pad_token = PAD_TOKEN):
        """
        The Pyramidal Bidiretional LSTM takes in a packed sequence of x, then
        1) pads the packed sequence
        2) reshapes the input from (batches x time x features) --> (batches x (time / 2) x (features * 2))
        3) pack the reshaped x
        4) run through bidirectional lstm
        """

        # Pad Packed Sequence
        x, lx = pad_packed_sequence(x_packed, batch_first=True, padding_value=pad_token)

        # Downsample X across time and Upsample across features
        x, lx = self.reshape_x(x, lx)

        # Pack x
        x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)

        # Input x through Bidirectional LSTM
        x, _ = self.lstm(x)

        return x

    def reshape_x(self, x, lx):
        # Extract features from x
        batch_size, max_len, features = x.shape

        # If x has an odd number of timepoints, remove the last one
        if max_len % 2 == 1:
            max_len = max_len - 1
        else:
            max_len = max_len

        x = x[:, :max_len, :]

        # Reshape x from (batches x time x features) --> (batches x (time / 2) x (features * 2))
        x = x.reshape(batch_size, int(max_len/2), int(2*features))

        # Clamp the input lx to the new max length
        lx = torch.clamp(lx, min = None, max = x.shape[1])

        return x, lx


class LockedDropout(torch.nn.Module):
    """
    LockedDropout applies the same dropout mask across all timesteps of an RNN during training
    """
    def __init__(self, dropout = 0.5, pad_token = PAD_TOKEN):
        super().__init__()
        self.dropout = dropout
        self.pad_token = pad_token

    def forward(self, x):
        if self.training and self.dropout > 0:
            x, lx = pad_packed_sequence(x, batch_first=True, padding_value=self.pad_token)

            # Create the mask in shape (batchsize x 1 x features)
            mask = x.data.new(x.shape[0], 1, x.shape[2]).bernoulli_(1 - self.dropout)
            # Scale the mask
            mask = mask / (1 - self.dropout)
            # Expand the mask to same shape as x
            mask = mask.expand_as(x)

            # Mask out x
            x = mask * x

            # Repack x
            x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)
            return x

        else:
            # Return the original array for testing or dropout = 0
            return x

class TransformerListener(torch.nn.Module):
    """
    pBLSTMS upsample the number of features before inputting through the bLSTM --> input size = 2x the normal input
    pBLSTMS are a bidirectional LSTM, so the output is twice as large as whatever is input -- > use 1/2 hidden
    """
    def __init__(self, input_size, listener_hidden_size, dropout, n_pBLSTM_layers = 1):
        super(TransformerListener, self).__init__()

        self.lstm = torch.nn.LSTM(input_size = input_size, hidden_size = listener_hidden_size // 2,
                                  num_layers = 1, bidirectional = True, batch_first = True)

        pBLSTMS = torch.nn.ModuleList([])
        for i in range(n_pBLSTM_layers):
            pBLSTMS.append(pBLSTM(input_size = 2 * listener_hidden_size, hidden_size = listener_hidden_size // 2))
            pBLSTMS.append(LockedDropout(dropout = dropout))

        self.pBLSTMs = torch.nn.Sequential(*pBLSTMS)

        # Downsample by factor of 8
        # However, this seems to require much more computational time than the pyramidal BLSTM (~4 hours/epoch vs. ~1.5 hours/epoch)
#         self.CDNNs =  torch.nn.Sequential(
#             torch.nn.Conv1d(in_channels = 2*listener_hidden_size, out_channels=listener_hidden_size, kernel_size=8, stride = 8),
#         )



    def forward(self, x, lx):
        """
        Take in x and lengths of x, then
        1) pack x
        2) pass x through the initial bidirectional lstm
        3) pass x through pyramidal bidirectional lstm
        4) pad x
        """
        x = pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)
        x, _ = self.lstm(x)
        x = self.pBLSTMs(x)

        # switch below to x, lx if using the pBLSTMs
        x, lx = pad_packed_sequence(x, batch_first=True)

        # To use the Conv layers, permute ... but we don't need to use the Conv bc we're using pBLSTM instead
#         x = x.permute(0, 2, 1)
#         x = self.CDNNs(x)
#         x = x.permute(0, 2, 1)
#         lx = (lx - 8) // 8

        return x, lx

In [24]:
class Attention(torch.nn.Module):

    def __init__(self, listener_hidden_size, speller_hidden_size, projection_size):
        super(Attention, self).__init__()

        self.KW = torch.nn.Linear(in_features = listener_hidden_size, out_features = projection_size)
        self.VW = torch.nn.Linear(in_features = listener_hidden_size, out_features = projection_size)
        self.QW = torch.nn.Linear(in_features = speller_hidden_size, out_features = projection_size)
        self.softmax = torch.nn.Softmax(dim = 1)

    def set_key_value(self, encoder_outputs, encoder_lens):

        # Calculate key and value
        self.key = self.KW(encoder_outputs)
        self.value = self.VW(encoder_outputs)

        # Create a mask for padded values
        max_len = encoder_outputs.shape[1]
        self.pad_mask = (torch.arange(max_len).unsqueeze(0) >= encoder_lens.unsqueeze(1)).to(device)

    def compute_context(self, decoder_context):
        self.query = self.QW(decoder_context)

        # Calculate raw weights
        # and mask out the ones corresponding to padding
        query_length = self.query.shape[1]
        raw_weights = np.sqrt(1/query_length) * torch.bmm(self.key, self.query.unsqueeze(2)).squeeze(2)
        raw_weights = raw_weights.masked_fill_(self.pad_mask, -float("inf"))

        # Calculate attention weights
        attention_weights = self.softmax(raw_weights)

        # Calculate context
        attention_context = torch.bmm(attention_weights.unsqueeze(1), self.value).squeeze(1)

        return attention_context, attention_weights

In [25]:
class Speller(torch.nn.Module):

    def __init__(self, embedding_size, lstm_hidden_size, lstm_output_size,
                 projection_size, attention, vocab_size=len(VOCAB), dropout = 0.2, weight_tying = True,
                 additional_lstm_layers = 0, max_timesteps = 600):
        """
        embed_size: the size of the input data to the Speller
        lstm_hidden_size: the size of hidden layers in the Speller
        lstm_output_size: the output size of the decoder LSTM
        projection_size: the size that vocab is projected to
        attention: the torch class used for attention
        vocab_size: the number of character classes for the final output
        dropout: the dropout rate for lstm cells
        weight_tying: boolean value for whether to do weight tying
        additional_lstm_layers: the number of additional lstm layers to include in the Speller


        """
        super().__init__()

        # Save self variables
        self.vocab_size  = vocab_size
        self.attend = attention
        self.projection_size = projection_size
        self.max_timesteps = max_timesteps

        # Embedding layer to convert token to latent space
        self.embedding = torch.nn.Embedding(num_embeddings = vocab_size,
                                                     embedding_dim = embedding_size,
                                                     padding_idx = PAD_TOKEN)

        # LSTM Cells
        # Note that the input size to the first LSTM cell is the projection size plus the embed size
        # First lstm layer
        lstm_cells = torch.nn.ModuleList([torch.nn.LSTMCell(input_size = projection_size + embedding_size,
                                                            hidden_size = lstm_hidden_size)])
        # Add any additional layers
        for i in range(additional_lstm_layers):
            lstm_cells.append(torch.nn.LSTMCell(input_size = lstm_hidden_size,
                                                hidden_size = lstm_hidden_size))
        # Add the output lstm layer
        lstm_cells.append(torch.nn.LSTMCell(input_size = lstm_hidden_size,
                                            hidden_size = lstm_output_size))

        # Convert the module list into Sequential
        self.lstm_cells = torch.nn.Sequential(*lstm_cells)

        # Create dropout layers for each lstm layer
        dropout_layers = [torch.nn.Dropout(p=dropout)] * len(self.lstm_cells)
        self.dropout_layers = torch.nn.ModuleList(dropout_layers)

        # Output to character
        # self.output_to_char
        self.output_to_char          = torch.nn.Linear(projection_size * 2, vocab_size)

        if weight_tying:
            # I think this only works if projection size = 1/2 embedding size
            assert projection_size * 2 == embedding_size, "Error! Cannot Perform Weight Tying with unequal sizes"
            self.output_to_char.weight   = self.embedding.weight # Weight tying

        self.decoder_output_size = lstm_output_size

    def lstm_step(self, input_word, hidden_state):
        """
        From hw4p1 - iterate through each cell, updating the hidden state for the current layer
        and input word for the next layer each time

        Apply dropout on the input word
        """
        for i in range(len(self.lstm_cells)):

#             print(i)
#             print("input_word: ", input_word.shape)
#             print("hidden: ", hidden_state[i][1].shape if hidden_state[i] is not None else "None")
            # Pass through layer of LSTM
            hidden_state[i] = self.lstm_cells[i](input_word, hidden_state[i])

            # Update embedding
            input_word = hidden_state[i][0]

            # Perform dropout on the output
            input_word =  self.dropout_layers[i](input_word)
            hidden_state[i] = (input_word, hidden_state[i][1])

        return input_word, hidden_state

    def forward(self, x, lx, y = None, teacher_forcing_ratio = 1):

        batch_size, max_seq_len, _ = x.shape

        attn_context = torch.zeros(batch_size, self.projection_size).to(device) # initial context tensor for time t = 0
        output_symbol = torch.full((batch_size,), fill_value=SOS_TOKEN, dtype=torch.long).to(device).long() # Set it to SOS for time t = 0
        raw_outputs = []
        attention_plot = []

        # Initialize attention parameters and context
        self.attend.set_key_value(x, lx)
        attention_weights = torch.zeros(batch_size, max_seq_len).to(device)


        if y is None:
          timesteps = self.max_timesteps
          teacher_forcing_ratio = 0 #Why does it become zero?

        else:
          timesteps = y.shape[1] # How many timesteps are we predicting for?
          label_embed = self.embedding(y)

        hidden_state = [None]*len(self.lstm_cells) # Initialize your hidden_states list here similar to HW4P1

        for t in range(timesteps):
    #       print("Timestep", t)
          p = np.random.random_sample() # generate a probability p between 0 and 1

          if p < teacher_forcing_ratio and t > 0 and self.training: # Why do we consider cases only when t > 0? What is considered when t == 0? Think.
             # Take from y, else draw from probability distribution
             # Can only take from y in training (shouldn't exist otherwise . . . )
            char_embed = label_embed[:, t-1, :]

          else:
             # Draw from the last output
             char_embed = self.embedding(output_symbol)

    #       print("output symbol: ", output_symbol.shape)
          # Embed the character symbol

          # Concatenate the character embedding and context from attention, as shown in the diagram

    #       print("char_embed: ", char_embed.shape)
    #       print("attn_context: ", attn_context.shape)
          lstm_input = torch.cat([char_embed, attn_context], dim = 1)

          input_word, hidden_state = self.lstm_step(lstm_input, hidden_state) # Feed the input through LSTM Cells and attention.
          # What should we retrieve from forward_step to prepare for the next timestep?


          final_hidden = hidden_state[-1][0]
          attn_context, attn_weights = self.attend.compute_context(final_hidden) # Feed the resulting hidden state into attention

          cdn_input = torch.cat([self.attend.query, attn_context], dim = 1) # TODO: You need to concatenate the context from the attention module with the LSTM output hidden state, as shown in the diagram

          raw_pred = self.output_to_char(cdn_input)

          # Generate a prediction for this timestep and collect it in output_symbols
          # Output symbol will be used for the next time step if not teacher forcing
          output_symbol = raw_pred.argmax(dim=1) # Draw correctly from raw_pred

          raw_outputs.append(raw_pred) # for loss calculation
          attention_plot.append(attn_weights)

        attention_plot = torch.stack(attention_plot, dim=1)
        raw_outputs = torch.stack(raw_outputs, dim=1)
        return raw_outputs, attention_plot

In [26]:
class ASRModel(torch.nn.Module):
    def __init__(self, input_size, listener_hidden_size,
                 vocab_size, embed_size,
                 speller_hidden_size, speller_output_size,
                 projection_size, dropout, n_pBLSTM_layers):
        """
        The model consists of the of the lister (an encoder), the attention module (which is paseed to the speller),
        and the speller (the decoder).
        """

        super(ASRModel, self).__init__()

        self.listener  = TransformerListener(input_size = input_size, listener_hidden_size=listener_hidden_size,
                                             dropout = dropout,
                                             n_pBLSTM_layers = n_pBLSTM_layers)

        self.attend = Attention(listener_hidden_size=listener_hidden_size,
                                speller_hidden_size=speller_output_size,
                                projection_size=projection_size)

        self.speller = Speller(embedding_size = embed_size,
                               lstm_hidden_size = speller_hidden_size,
                               lstm_output_size = speller_output_size,
                               vocab_size = vocab_size,
                               projection_size = projection_size,
                               attention = self.attend,
                               dropout = dropout)

    def forward(self, x, lx, y = None, teacher_forcing_ratio = 1):
        # Peform any transforms to x
        if self.training:
          x = x.permute(0, 2, 1)
          x = transforms(x)
          x = x.permute(0, 2, 1)

        encoder_outputs, lx = self.listener(x, lx)
        raw_outputs, attention_plots = self.speller(encoder_outputs, lx, y, teacher_forcing_ratio)
        return raw_outputs, attention_plots

In [27]:
torch.cuda.empty_cache()

model = ASRModel(input_size = 62, listener_hidden_size = 128, vocab_size = len(VOCAB),
                 embed_size = 2048, speller_hidden_size = 128, speller_output_size = 64,
                 projection_size = 1024, dropout = 0.18, n_pBLSTM_layers = 3)

model = model.to(device)
print(model)

ASRModel(
  (listener): TransformerListener(
    (lstm): LSTM(62, 64, batch_first=True, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (lstm): LSTM(256, 64, batch_first=True, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (lstm): LSTM(256, 64, batch_first=True, bidirectional=True)
      )
      (3): LockedDropout()
      (4): pBLSTM(
        (lstm): LSTM(256, 64, batch_first=True, bidirectional=True)
      )
      (5): LockedDropout()
    )
  )
  (attend): Attention(
    (KW): Linear(in_features=128, out_features=1024, bias=True)
    (VW): Linear(in_features=128, out_features=1024, bias=True)
    (QW): Linear(in_features=64, out_features=1024, bias=True)
    (softmax): Softmax(dim=1)
  )
  (speller): Speller(
    (attend): Attention(
      (KW): Linear(in_features=128, out_features=1024, bias=True)
      (VW): Linear(in_features=128, out_features=1024, bias=True)
      (QW): Linear(in_features=64, out_features=1024, bias=Tru

In [28]:
criterion   = torch.nn.CrossEntropyLoss(ignore_index=VOCAB_MAP['<pad>'])

optimizer   = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], amsgrad=True, weight_decay=config['weight_decay'])  # TODO: Define the optimizer. Adam/AdamW usually works good for this HW

scheduler   = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4) # Optional TODO: Use a scheduler if you want

scaler      = torch.cuda.amp.GradScaler() # TODO

In [29]:
# We have given you this utility function which takes a sequence of indices and converts them to a list of characters
def indices_to_chars(indices, vocab):
    tokens = []
    for i in indices: # This loops through all the indices
        if int(i) == SOS_TOKEN: # If SOS is encountered, dont add it to the final list
            continue
        elif int(i) == EOS_TOKEN: # If EOS is encountered, stop the decoding process
            break
        else:
            tokens.append(vocab[i])
    return tokens

# To make your life more easier, we have given the Levenshtein distantce / Edit distance calculation code
def calc_edit_distance(predictions, y, y_len, vocab= VOCAB, print_example= False, batch_num = -1):

    dist                = 0
    batch_size, seq_len = predictions.shape

    for batch_idx in range(batch_size):

        y_sliced    = indices_to_chars(y[batch_idx,0:y_len[batch_idx]], vocab)
        pred_sliced = indices_to_chars(predictions[batch_idx], vocab)

        # Strings - When you are using characters from the AudioDataset
        y_string    = ''.join(y_sliced)
        pred_string = ''.join(pred_sliced)

        #dist        += Levenshtein.distance(pred_string, y_string)
        # Comment the above abd uncomment below for toy dataset
        dist      += Levenshtein.distance(y_sliced, pred_sliced)

    if (print_example) and (batch_num == 0):
        # Print y_sliced and pred_sliced if you are using the toy dataset
        print("\nGround Truth : ", y_string)
        print("Prediction   : ", pred_string)

    dist    /= batch_size
    return dist


def train(model, dataloader, criterion, optimizer, teacher_forcing_rate):

    model.train()
    batch_bar = blue_tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    running_loss        = 0.0
    running_perplexity  = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):

        optimizer.zero_grad()

        x, y, lx, ly = x.to(device), y.to(device), lx, ly

        with torch.cuda.amp.autocast():

            raw_predictions, attention_plot = model(x, lx, y= y, teacher_forcing_ratio = teacher_forcing_rate)

            # Predictions are of Shape (batch_size, timesteps, vocab_size).
            # Transcripts are of shape (batch_size, timesteps) Which means that you have batch_size amount of batches with timestep number of tokens.
            # So in total, you have batch_size*timesteps amount of characters.
            # Similarly, in predictions, you have batch_size*timesteps amount of probability distributions.
            # How do you need to modify transcipts and predictions so that you can calculate the CrossEntropyLoss? Hint: Use Reshape/View and read the docs
            # Also we recommend you plot the attention weights, you should get convergence in around 10 epochs, if not, there could be something wrong with
            # your implementation

            # loss function has been set to ingnore padding index
            # print("Pred shape: ", raw_predictions.shape, raw_predictions.data.shape)
            # print("Y shape: ", y.shape, y.data.shape)

            loss        =  criterion(raw_predictions.view(-1, raw_predictions.shape[2]), y.view(-1,)) # TODO: Cross Entropy Loss

            perplexity  = torch.exp(loss) # Perplexity is defined the exponential of the loss
            running_loss        += loss.item()
            running_perplexity  += perplexity.item()

        # Backward on the masked loss
        scaler.scale(loss).backward()
        # loss.backward()

        # Optional: Use torch.nn.utils.clip_grad_norm to clip gradients to prevent them from exploding, if necessary
        # If using with mixed precision, unscale the Optimizer First before doing gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

        scaler.step(optimizer)
        scaler.update()


        batch_bar.set_postfix(
            loss="{:.04f}".format(running_loss/(i+1)),
            perplexity="{:.04f}".format(running_perplexity/(i+1)),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
            tf_rate='{:.02f}'.format(teacher_forcing_rate))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    running_loss /= len(dataloader)
    running_perplexity /= len(dataloader)
    batch_bar.close()

    return running_loss, running_perplexity, attention_plot

def validate(model, dataloader):

    model.eval()

    batch_bar = blue_tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc="Val")

    running_lev_dist = 0.0

    for i, (x, y, lx, ly) in enumerate(dataloader):

        x, y, lx, ly = x.to(device), y.to(device), lx, ly

        with torch.inference_mode():
            raw_predictions, attentions = model(x, lx, y = None)

        # Greedy Decoding
        greedy_predictions   = torch.argmax(raw_predictions, dim = 2) # TODO: How do you get the most likely character from each distribution in the batch?

        # Calculate Levenshtein Distance
        running_lev_dist    += calc_edit_distance(greedy_predictions, y, ly, VOCAB, print_example = True,
                                                  batch_num = i) # You can use print_example = True for one specific index i in your batches if you want

        batch_bar.set_postfix(
            dist="{:.04f}".format(running_lev_dist/(i+1)))
        batch_bar.update()

        del x, y, lx, ly
        torch.cuda.empty_cache()

    batch_bar.close()
    running_lev_dist /= len(dataloader)

    return running_lev_dist

EEG to phonemes

input EEG --> output phoneme

EEG = (12 * N_subjects,) varying length

"labels" = list (12 * N_subjects,) varying length


Follow HW3P2 logic

Padpacked sequence whatever dataloader

In [30]:
import wandb
wandb.login(key="dd406b9c9216ffc7053f08e284381c34b4b95df6")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [31]:
run = wandb.init(
    name = "Attention-Model-Alpha", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_group_project", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: jkosnoff (idl-f23-bestteam). Use `wandb login --relogin` to force relogin


In [32]:
torch.cuda.empty_cache()
gc.collect()

5778

In [ ]:
torch.cuda.empty_cache()
gc.collect()

best_lev_dist = float("inf")
tf_rate = 1.0
path = r"./checkpoint.pth"
tf_scheduler = None

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    # Call train and validate, get attention weights from training
    running_loss, running_perplexity, attention_plot = train(model, train_loader, criterion, optimizer, tf_rate)

    # Print your metrics
    print('[TRAIN] \t Loss: %.4f \t Perplexity: %.4f \tLr: %.6f'
          % (running_loss, running_perplexity, optimizer.param_groups[0]['lr']))


    running_lev_dist = validate(model, val_loader)
    print('[VAL] \tLoss: %.4f'% (running_lev_dist))

    # Plot Attention for a single item in the batch
    plot_attention(attention_plot[0].detach().cpu())

    # Log metrics to Wandb
    wandb.log({
        'train_loss': running_loss,
        'train_perplex': running_perplexity,
        'valid_dist': running_lev_dist,
        'lr'        : optimizer.param_groups[0]['lr'],
    })

    # Optional: Scheduler Step / Teacher Force Schedule Step
    scheduler.step(running_lev_dist)

    if running_lev_dist <= best_lev_dist:
        best_lev_dist = running_lev_dist
        metric = ["valid_dist", running_lev_dist]
        save_model(model, optimizer, scheduler, tf_scheduler, metric, epoch, path)
        wandb.save(os.path.join(wandb.run.dir, "checkpoint*"))
        print("Saved best model")
        # Save your model checkpoint here

Output hidden; open in https://colab.research.google.com to view.